# <center> ERP Type Classification </center>
## Binary classification of intracranial data into trial type.

*Analysis by David Huberdeau*

### Background

Patients with epilepsy who were implanted with intracranial electrodes (depth probes and/or grids) participated in an experiment that had them reaching to different targets presented on a screen (a visuomotor association task). Trials either had no-cue, a direct cue, or a symbolic cue. Signals from the EEG record were recorded during this task, and epochs of the recordings were taken around the time that the cue was presented, as well as the time that the target was presented. Is there a difference in response in the hippocampus when a cue was symbolic compared to no-cue, or when there was a direct cue compared to no cue?

### Approach

In many cases, multiple electrodes recorded the hippocampal activity. Some electrodes might be better than others are detecting a difference, and the combined activity might have more information than any one alone, or than their activity averaged together.

Here, I will use a decision tree [and possibly other classification and dimensionality methods] to determine if the signals differ between no-cue and the other two cue conditions, and also use it to determine which electrodes are most important for the differentiation.


### Results

A tree classifier differentiates hippocampal signals between Symbolic trials and no-cue trials (approx. 65% accuracy, where 50% is chance), while it does not differentiate Direct  trials from no-cue trials (approx. 50% accuracy). 

The electrode importance scores were discovered and saved to be used for further analyses.

See accompanying Matlab code (`review_tree_classifier_outcomes.m`) for presentation of some of these analyses.



### Conclusions

Hippocampal activity significantly differed in response to the presentation of a symbolic cue, compared to the presentation of a direct cue. This is consistent with the hippocampus' role in recall of associations.



In [1]:
# Load necessary software packages
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import scipy.io
import os
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import train_test_split as split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder as enc
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import ComplementNB
# from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn import datasets
from sklearn import metrics
from scipy.sparse import csr_matrix, hstack

In [2]:
# Define some parameters:
MAX_DEPTH = 5
MAX_LEAF_NODES = 5
N_ESTIMATORS = 100

In [3]:
# Define where the data lives:
data_dir = '/Users/david/Box Sync/projects/vma_ieeg_combined/intermediate_data'

sub_data = ['LL__erp_data.mat', 
           'DO__erp_data.mat',
           'JB__erp_data.mat',
           'BP__erp_data.mat',
           'VS__erp_data.mat',
           'JK__erp_data.mat',
           'LO__erp_data.mat',
           'NH__erp_data.mat']

labels_data = ['LL_labels.mat', 
           'DO_labels.mat',
           'JB_labels.mat',
           'BP_labels.mat',
           'VS_labels.mat',
           'JK_labels.mat',
           'LO_labels.mat',
           'NH_labels.mat']

category_data = 'rt_set_vma1.mat'

sub_channels = [
    ['LH1', 'LH2', 'LH3', 'RH1', 'RH2', 'RH3'],
    ['II1', 'II2', 'II3', 'HH1', 'HH2'],
    ['SS1', 'SS2', 'II1', 'II2'],
    ['MM1', 'MM2', 'MM3', 'MM4'],
    ['EE1', 'FF1', 'FF2'],
    ['HH1', 'HH2', 'HH3', 'HH4', 'HH5', 'KK1', 'KK2'],
    ['KK1', 'KK2', 'MM1', 'MM2', 'MM3', 'OO1', 'OO2'],
    ['DD1', 'DD2', 'DD3'],
]

In [4]:
# Define functions for loading the data:
def load_features(data_dir, sub_data):
    factor_mat_ = scipy.io.loadmat(data_dir + os.sep + sub_data)
    factor_mat_ = factor_mat_['features']
    factor_mat = np.transpose(factor_mat_)
    return factor_mat

def load_labels(data_dir, labels_data):
    labels_mat = scipy.io.loadmat(data_dir + os.sep + labels_data)
    labels_mat_ = labels_mat['data'][0][0][0][0]
    labels_out = []
    for i_lab in range(len(labels_mat_)):
        labels_out.append(labels_mat_[i_lab][0])
    return labels_out

def load_behavior(data_dir, category_data, subject_num):
    cat_mat = scipy.io.loadmat(data_dir + os.sep + category_data)
    cat_mat_ = cat_mat['rt_set'][0][subject_num]
    rt = []
    ty = []
    for i_row in range(len(cat_mat_)):
        rt.append(cat_mat_[i_row][0])
        ty.append(cat_mat_[i_row][1])
    return rt, ty
    
def get_channel_index(labels, channel_label):
    for i_ch in range(len(labels)):
        if (labels[i_ch] == channel_label):
            k_ch = i_ch
    return k_ch 

def get_channel_features(features, labels, channel_labels):
    k_inds = []
    for i_ch in range(len(channel_labels)):
        k_inds.append(get_channel_index(labels, channel_labels[i_ch]))
    sub_features_ = features[k_inds]
    sub_features = np.transpose(sub_features_)
    return sub_features

def order_channels_by_importance(channel_list, importance_list):
    # display the relative importance of each attribute
    f_imp = importance_list
    sort_order_ = np.argsort(f_imp)
    sort_order = list(reversed(sort_order_))
    channels_sorted = []
    importance_sorted = []
    for i_fact in range(0,len(sort_order)):
        channels_sorted.append(channel_list[sort_order[i_fact]])
        importance_sorted.append(f_imp[sort_order[i_fact]])
    return channels_sorted, importance_sorted

def save_outputs(acc_1, acc_2, ch_sort_1, ch_sort_2, score_sort_1, score_sort_2):
    scipy.io.savemat('tree_accuracy_1.mat', {'acc_':acc_1})
    scipy.io.savemat('tree_accuracy_2.mat', {'acc_':acc_2})
    scipy.io.savemat('channel_sort_1.mat', {'channel_sort_':ch_sort_1})
    scipy.io.savemat('channel_sort_2.mat', {'channel_sort_':ch_sort_2})
    scipy.io.savemat('score_sort_1.mat', {'score_sort_':score_sort_1})
    scipy.io.savemat('score_sort_2.mat', {'score_sort_':score_sort_2})


In [5]:
labels = load_labels(data_dir, labels_data[0])
k_ind = get_channel_index(labels, 'LH1')
print(k_ind)
print(labels[k_ind])

24
LH1


In [6]:
# cat_mat = scipy.io.loadmat(data_dir + os.sep + category_data)
# cat_mat['rt_set'][0][3]

rt, ty = load_behavior(data_dir, category_data, 4)


In [7]:
features = load_features(data_dir, sub_data[0])
labels = load_labels(data_dir, labels_data[0])
f = get_channel_features(features, labels, ['LH1', 'LH2'])
print(np.shape(f))

(182, 2)


In [10]:
# For each subject, load in the data, which is in matlab format (.mat)
f_clf_acc_1 = np.empty(len(labels_data))
f_imp_sub_1 = np.empty(len(labels_data), dtype = object)
f_chs_sub_1 = np.empty(len(labels_data), dtype = object)
f_clf_acc_2 = np.empty(len(labels_data))
f_imp_sub_2 = np.empty(len(labels_data), dtype = object)
f_chs_sub_2 = np.empty(len(labels_data), dtype = object)
for i_sub in range(len(labels_data)):
    feature_mat = load_features(data_dir, sub_data[i_sub])
    labels_mat = load_labels(data_dir, labels_data[i_sub])
    rt_mat_, ty_mat_ = load_behavior(data_dir, category_data, i_sub)
    rt_mat = np.array(rt_mat_)
    ty_mat = np.array(ty_mat_)
    sub_feature = get_channel_features(feature_mat, labels_mat, sub_channels[i_sub])
#     print(str(i_sub) + ': ' + str(np.shape(sub_feature)) + ': ' + str(np.shape(rt_mat)) + ': ' + str(np.shape(ty_mat)))
    
    # classify direct from no-cue:
    ty_0_1 = [ty_mat[x] == 0 or ty_mat[x] == 1 for x in range(len(ty_mat))]
    sfeat_0_1 = sub_feature[ty_0_1]
    stype_0_1 = ty_mat[ty_0_1]
    F_train, F_test, Y_train, Y_test = split(sfeat_0_1, stype_0_1, test_size = .25)
#     model = AdaBoostClassifier(n_estimators=N_ESTIMATORS, random_state=0)
    model = ExtraTreesClassifier(n_estimators=N_ESTIMATORS, random_state=0, max_depth=MAX_DEPTH)
#     model = GaussianNB()
    model.fit(F_train, Y_train)
    # Compute the importance of each attribute
    f_imp = model.feature_importances_
    [chs_sorted, imp_sorted] = order_channels_by_importance(sub_channels[i_sub], f_imp)
    f_imp_sub_1[i_sub] = imp_sorted
    f_chs_sub_1[i_sub] = chs_sorted
    # Compute the classification score
    f_score = model.score(F_test, Y_test)
    f_clf_acc_1[i_sub] = f_score

    # classify symbolic from no-cue:
    ty_0_2 = [ty_mat[x] == 0 or ty_mat[x] == 2 for x in range(len(ty_mat))]
    sfeat_0_2 = sub_feature[ty_0_2]
    stype_0_2 = ty_mat[ty_0_2]
    F_train, F_test, Y_train, Y_test = split(sfeat_0_2, stype_0_2, test_size = .25)
#     model = AdaBoostClassifier(n_estimators=N_ESTIMATORS, random_state=0)
    model = ExtraTreesClassifier(n_estimators=N_ESTIMATORS, random_state=0, max_depth=MAX_DEPTH)
#     model = GaussianNB()
#     model = ComplementNB()
    model.fit(F_train, Y_train)
    # Compute the importance of each attribute
    f_imp = model.feature_importances_
    [chs_sorted, imp_sorted] = order_channels_by_importance(sub_channels[i_sub], f_imp)
    f_imp_sub_2[i_sub] = imp_sorted
    f_chs_sub_2[i_sub] = chs_sorted
    # Compute the classification score
    f_score = model.score(F_test, Y_test)
    f_clf_acc_2[i_sub] = f_score
    
    
    
# print(f_imp_sub_1)
# print(f_imp_sub_2)
# print(f_chs_sub_1)
# print(f_chs_sub_2)
save_outputs(f_clf_acc_1, f_clf_acc_2, f_chs_sub_1, f_chs_sub_2, f_imp_sub_1, f_imp_sub_2)
print('Type1 accuracy: ' + str(np.mean(f_clf_acc_1)))
print('Type2 accuracy: ' + str(np.mean(f_clf_acc_2)))

Type1 accuracy: 0.5231182795698924
Type2 accuracy: 0.6423804226918799
